In [ ]:
!pip install kaggle
!pip install lightgbm

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import boto3
import os
import random
from datetime import date, datetime, timedelta
import gc
from io import StringIO

import lightgbm as lgb

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

BUCKET = 'dtci-dataplatform-telemetry-datsci-dev-bucket'
S3_PATH = 'Jason/m5'
CLIENT = boto3.client('s3')

In [6]:
dataset_start = 1500
dataset_end = 1913
nrows = None

numcols = [f"d_{day}" for day in range(dataset_start, dataset_end + 1)]
#catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
catcols = ['id']
dtype = {numcol:"float32" for numcol in numcols} 
dtype.update({col: "category" for col in catcols if col != "id"}) 

filename = 'sales_train_validation.csv'
obj = CLIENT.get_object(Bucket = BUCKET, Key = os.path.join(S3_PATH, filename))
df = pd.read_csv(obj['Body'], nrows = nrows, usecols = catcols + numcols, dtype = dtype)

filename = 'calendar.csv'
obj = CLIENT.get_object(Bucket = BUCKET, Key = os.path.join(S3_PATH, filename))
cal =  pd.read_csv(obj['Body'], dtype = CAL_DTYPES)
cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")
        cal[col] -= cal[col].min()

### Log Transform

In [7]:
def get_timespan(df, end, periods, freq='D'):
    """
    Returns subset of dataframe sliced by a given range of dates (start : start + periods)
    """
    return df[pd.date_range(end=end, periods=periods, freq=freq)]

In [8]:
# set index to id
df = df.set_index('id')
# Map codes to actual dates
d_to_date = dict(zip(cal['d'], cal['date']))
date_to_d = dict(zip(cal['date'], cal['d']))
df = df.rename(columns=d_to_date)

In [9]:
train_y_start = [1858, 1851, 1844, 1837, 1830, 1823, 1816]
valid_y_start = 1886
test_y_start = 1914

# convert to datetime
train_y_start = [d_to_date['d_{}'.format(integer_date)] for integer_date in train_y_start]
valid_y_start = d_to_date['d_{}'.format(valid_y_start)]
test_y_start = d_to_date['d_{}'.format(test_y_start)]

### Prepare Dataset

In [10]:
def prepare_dataset(df, y_start, is_train = True):
    # the X_start is 1 day before y_start
    X_start = y_start - pd.Timedelta(days=1)
    
    X = {}
    
    for i in [1, 3, 7, 14, 30, 60, 140]:
        # aggregate total
        X['agg_sales_{}_days'.format(i)] = get_timespan(df, X_start, i).sum(axis=1)
        
        # diff mean, exp decay, mean, median, min, max, std
        tmp = get_timespan(df, X_start, i)
        X['diff_mean_sales_{}_days'.format(i)] = tmp.diff(axis=1).mean(axis=1).values
        X['exp_decay_sales_{}_days'.format(i)] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_sales_{}_days'.format(i)] = tmp.mean(axis=1).values
        X['median_{}_days'.format(i)] = tmp.median(axis=1).values
        X['min_{}_days'.format(i)] = tmp.min(axis=1).values
        X['max_{}_days'.format(i)] = tmp.max(axis=1).values
        X['std_{}_days'.format(i)] = tmp.std(axis=1).values
        
        # diff mean, exp decay, mean, median, min, max, std lagged by 1 week
        tmp = get_timespan(df, X_start - pd.Timedelta(days = 7), i)
        X['diff_mean_sales_1weeklag_{}_days'.format(i)] = tmp.diff(axis=1).mean(axis=1).values
        X['exp_decay_sales_1weeklag_{}_days'.format(i)] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_sales_1weeklag_{}_days'.format(i)] = tmp.mean(axis=1).values
        X['median_1weeklag_{}_days'.format(i)] = tmp.median(axis=1).values
        X['min_1weeklag_{}_days'.format(i)] = tmp.min(axis=1).values
        X['max_1weeklag_{}_days'.format(i)] = tmp.max(axis=1).values
        X['std_1weeklag_{}_days'.format(i)] = tmp.std(axis=1).values
        
        # mean + exponential decay number of sales for days when there was at least one sale
        tmp1 = get_timespan(df, X_start, i)
        tmp2 = (get_timespan(df, X_start, i) > 0) * 1
        X['mean_nonzero_sales_{}_days'.format(i)] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['exp_decay_nonzero_sales_{}_days'.format(i)] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        
        # mean + exponential decay number of days when there wasn't a sale
        X['mean_number_of_days_with_no_sales_{}_days'.format(i)] = (1 - tmp2).mean(axis=1).values
        X['exp_decay_number_of_days_with_no_sales_{}_days'.format(i)] = ((1 - tmp2) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        
        # last sale day
        tmp = get_timespan(df, X_start, i)
        X['has_sale_in_past_{}_days'.format(i)] = (tmp > 0).sum(axis=1).values
        X['days_since_last_sale_{}_days'] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values # if no sale in the past i days, return i
        X['weighted_days_since_last_sale_{}_days'] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values
    
    for i in range(1, 16):
        # actual sales
        X['actual_number_of_sales_{}_days'.format(i)] = get_timespan(df, X_start - pd.Timedelta(days=i), 1).values.ravel()
    
    for i in range(7):
        X['mean_DOW_{}_past_4_weeks'.format(i)] = get_timespan(df, X_start - pd.Timedelta(days=28-i), 4, freq='7D').mean(axis=1).values
        X['mean_DOW_{}_past_20_weeks'.format(i)] = get_timespan(df, X_start - pd.Timedelta(days=140-i), 20, freq='7D').mean(axis=1).values
    
    X = pd.DataFrame(X)
    
    if is_train:
        y = df[
            pd.date_range(y_start, periods=28)
        ].values
        return X, y
    
    else:
        return X
    

In [11]:
X_1, y_1 = [], []
for y_start in train_y_start:
    X_tmp, y_tmp = prepare_dataset(df, y_start)
    X_1.append(X_tmp)
    y_1.append(y_tmp)

X_train = pd.concat(X_1, axis=0)
y_train = np.concatenate(y_1, axis=0)

del X_1, y_1; gc.collect()

232

In [12]:
X_val, y_val = prepare_dataset(df, valid_y_start)

In [13]:
X_test = prepare_dataset(df, test_y_start, is_train=False)

### Train Model
Train one model for each day in y (28 models total)

### Model Parameters

In [14]:
params = {
    'num_leaves': 80,
    'objective': 'poisson',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'rmse',
    'num_threads': 4
}

In [15]:
MAX_ROUNDS = 10000
val_pred = []
test_pred = []

# placeholder for categorical variables
cate_vars = []

In [16]:
for i in range(28):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    
    # Instantiate Dataset
    dtrain = lgb.Dataset(X_train.values, label=y_train[:, i], categorical_feature=cate_vars)
    dval = lgb.Dataset(X_val.values, label=y_val[:, i], reference=dtrain, categorical_feature=cate_vars)
    
    # Train Model
    bst = lgb.train(params, dtrain, num_boost_round=MAX_ROUNDS, 
                    valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=MAX_ROUNDS)
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

Step 1


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[189]	training's rmse: 1.95727	valid_1's rmse: 1.76406
Step 2
Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[249]	training's rmse: 1.73667	valid_1's rmse: 1.85046
Step 3
Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[202]	training's rmse: 1.85803	valid_1's rmse: 1.6936
Step 4
Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[213]	training's rmse: 1.86609	valid_1's rmse: 1.81295
Step 5
Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[538]	training's rmse: 1.78188	valid_1's rmse: 2.07575
Step 6
Training until validation scores don't improve for 125 rounds
Early stopping, best iteration is:
[499]	training's rmse: 2.15472	valid_1's rmse: 2.30095
Step 7
Training until validation scores don't improve 

In [17]:
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Validation mse: 4.380480390569589


### Submission

In [18]:
submission = pd.DataFrame(data=np.array(test_pred).transpose(),  index=X_test.index)
submission.columns = ['F{}'.format(i + 1) for i in submission.columns.values]
submission.reset_index(inplace=True)

# dummy data for evaluation
submission_eval = submission.copy()
submission_eval["id"] = submission_eval["id"].str.replace("validation$", "evaluation")
submission = pd.concat([submission, submission_eval], axis=0, sort=False)



In [19]:
filename = 'submission_sequential_lgbm_{}.csv'.format(date.today().strftime('%m%d_%h'))
csv_buffer = StringIO()
submission.to_csv(csv_buffer, index=False)
s3_object = boto3.resource('s3').Object(BUCKET, os.path.join(S3_PATH, 'submissions', filename))
s3_object.put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'B801D68F8A2B002D',
  'HostId': 'xWu2Lgewkhibv+7YFIqYT7WfsjteRR2olPhQB4y1ZrG5sEkmt5wfeFu2MHt11KNV5aS15FybCGM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xWu2Lgewkhibv+7YFIqYT7WfsjteRR2olPhQB4y1ZrG5sEkmt5wfeFu2MHt11KNV5aS15FybCGM=',
   'x-amz-request-id': 'B801D68F8A2B002D',
   'date': 'Sat, 18 Apr 2020 20:07:58 GMT',
   'etag': '"c38ef18cc2f95761f8bf299d06bafbcf"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"c38ef18cc2f95761f8bf299d06bafbcf"'}